In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../extraction_core/page_selection")
sys.path.append("../extraction_core/extraction")
sys.path.append("../extraction_core/")

In [4]:
import hvac
import os
client = hvac.Client(
    url="https://vault.lab.sspcloud.fr", token=os.environ["VAULT_TOKEN"]
)

secret = os.environ["VAULT_MOUNT"] + os.environ["VAULT_TOP_DIR"] + "/s3_creds"
mount_point, secret_path = secret.split("/", 1)
secret_dict = client.secrets.kv.read_secret_version(
    path=secret_path, mount_point=mount_point
)

os.environ["AWS_ACCESS_KEY_ID"] = secret_dict["data"]["data"][
    "ACCESS_KEY"
]
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_dict["data"]["data"][
    "SECRET_KEY"
]
try:
    del os.environ["AWS_SESSION_TOKEN"]
except KeyError:
    pass

In [5]:
from extraction.data import fs

fs.ls("projet-extraction-tableaux/orange/bmp")

['projet-extraction-tableaux/orange/bmp/306769688.bmp',
 'projet-extraction-tableaux/orange/bmp/307299248.bmp',
 'projet-extraction-tableaux/orange/bmp/345039416.bmp',
 'projet-extraction-tableaux/orange/bmp/379984891.bmp',
 'projet-extraction-tableaux/orange/bmp/380129866.bmp',
 'projet-extraction-tableaux/orange/bmp/384518114.bmp',
 'projet-extraction-tableaux/orange/bmp/409759156.bmp',
 'projet-extraction-tableaux/orange/bmp/430107359.bmp',
 'projet-extraction-tableaux/orange/bmp/432668432.bmp',
 'projet-extraction-tableaux/orange/bmp/440419240.bmp',
 'projet-extraction-tableaux/orange/bmp/500413505.bmp',
 'projet-extraction-tableaux/orange/bmp/500440813.bmp',
 'projet-extraction-tableaux/orange/bmp/501614572.bmp']

In [6]:
from pathlib import Path

for path in fs.ls("projet-extraction-tableaux/orange/bmp"):
    file_name = Path(path).stem
    fs.get(path, "../data/orange/" + file_name + ".bmp")

In [7]:
from PIL import Image
from extraction.table_extractor import TableExtractor
import numpy as np

In [8]:
table_extractor = TableExtractor.from_checkpoint(
    "projet-extraction-tableaux/logs/TableNetModule/version_00/checkpoints/marmot_model.ckpt",
    config="--oem 3 --psm 11"
)

siren_list = [
    "306769688",
    "307299248",
    "345039416",
    "379984891",
    "380129866",
    "384518114",
    "409759156",
    "430107359",
    "432668432",
    "440419240",
    "500413505",
    "500440813",
    "501614572"
]
data_dir = "../data/orange"
test_images = [
    Path(data_dir).joinpath(path + ".bmp") for path in siren_list
]

extracted_tables = {}
clean_tables = {}

for siren, image_path in zip(siren_list, test_images):
    extracted_tables[siren] = []
    image = Image.open(image_path)
    out = table_extractor.extract(image)

    for i, df in enumerate(out["tables"]):
        extracted_tables[siren].append(df)

/opt/mamba/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:200: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.1.6 to v2.0.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file weights.ckpt`
/opt/mamba/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/mamba/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments oth

In [9]:
for siren in siren_list:
    print("---")
    print(siren)
    print(len(extracted_tables[siren]))

---
306769688
1
---
307299248
1
---
345039416
2
---
379984891
4
---
380129866
1
---
384518114
1
---
409759156
1
---
430107359
2
---
432668432
2
---
440419240
1
---
500413505
2
---
500440813
2
---
501614572
2


In [10]:
fields = [
    "Quote-part du capital détenue (en %)",
    "Valeur brute comptable des titres détenus (en K euros)",
    "Valeur nette comptable des titres détenus (en K euros)",
    "Dividendes encaissés par la société en cours d'exercice (en K euros)",
    "Résultats (bénéfices ou pertes à 100%) "
    "du dernier exercice clos (en K devises)",
]

# Liste des pattern regex afin de déterminer les colonnes d'intérêt
regexes = [
    r"\b(quote-part){e<2}\b",
    r"\b(brute){e<2}\b",
    r"\b(nette){e<2}\b",
    r"\b(dividende){e<4}\b",
    r"\b(resultat){e<3}\b",
]

In [61]:
def boolean_list_densities(boolean_list, window_size=5):
    densities = []

    for i in range(total_length - window_size + 1):
        window = boolean_list[i:i+window_size]
        current_density = sum(window) / window_size
        densities.append(current_density)

    return densities

In [76]:
def count_words_in_row(row, regexes):
    count = 0
    for regex_pattern in regexes:
        for cell_value in row:
            if isinstance(cell_value, str):
                if regex.search(
                    regex_pattern, cell_value, flags=regex.IGNORECASE
                ):
                    count += 1
    return count

In [142]:
from typing import List
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
import regex


class NewTableCleaner:
    """
    Cleaner for the table output by `TableExtractor.extract`.
    """

    def __init__(
        self,
        colnames_out: List[str],
        regexes: List[str],
        pct_digit: float = 0.5,
    ):
        """
        Constructor for the TableCleaner class.

        Args:
            colnames_out (List[str]): Columns desired in the output.
            regexes (List[str]): Regular expressions to find in column names.
            pct_digit (float): Minimum percentage of digits in a row (with
                the first column removed) to have it considered as numeric.
        """
        self.colnames_out = colnames_out
        self.regexes = regexes
        self.pct_digit = pct_digit

    def clean_df(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Selects relevant columns in the pd.DataFrame given as input and
        returns a filtered pd.DataFrame with a clean index and clean
        column names.

        Args:
            df (pd.DataFrame): DataFrame to clean.
        """
        df_without_first_column = df.iloc[:, 1:]
        row_types = self.get_row_types(df_without_first_column)

        if not row_types:
            return df

        # First row can never be values already
        row_types[0] = False
        print("---")
        print(siren)

        if not any(row_types):
            header_rows = np.arange(len(row_types))
            body_rows = np.array([])
        else:
            true_index= 0
            while True and (true_index < len(np.where(row_types)[0])):
                print(true_index)
                header_length = np.where(row_types)[0][true_index]
                header_rows = np.arange(header_length)
                body_rows = np.arange(header_length, df.shape[0])
                word_counts = df.iloc[header_rows].apply(count_words_in_row, axis=1, regexes=self.regexes).sum()
                print(word_counts)
                if word_counts == 0:
                    print("hi")
                    true_index += 1
                else:
                    break

        header_df = df.iloc[header_rows]
        body_df = df.iloc[body_rows]

        similarities = []
        fuzzy_matches = []

        for name, value in header_df.items():
            value_cleaned = value.dropna()
            content = " ".join(value_cleaned)
            similarities.append(
                np.array(
                    [
                        self.similar(content, field)
                        for field in self.colnames_out
                    ]
                )
            )
            fuzzy_matches.append(
                np.array(
                    [
                        1
                        if regex.search(
                            regex_pattern, content, flags=regex.IGNORECASE
                        )
                        else 0
                        for regex_pattern in self.regexes
                    ]
                )
            )

        similarities = np.stack(similarities)
        fuzzy_matches = np.stack(fuzzy_matches)

        N, M = similarities.shape
        # Arrays of shape (N, M) with N the number of columns
        # in the extracted table and M = 6 (number of columns we
        # want to match)

        if N > M:
            column_ids = self.ids_from_long_similarities(similarities, fuzzy_matches)
        else:
            column_ids = self.ids_from_wide_similarities(similarities, fuzzy_matches)

        body_df.columns = column_ids
        
        return body_df

    def get_row_types(self, df: pd.DataFrame) -> List[bool]:
        """
        From an input DataFrame, returns a list of booleans and of
        length the number of rows in the DataFrame. Each element of this
        list is set to True if the corresponding row has a percentage of
        digit characters greater than pct_digit and False otherwise.

        Args:
            df (pd.DataFrame): DataFrame to get row types for.
        """
        row_types = []

        for row_index, row in df.iterrows():
            row_cleaned = row.dropna()
            row_as_string = "".join(row_cleaned)

            numbers = sum(c.isdigit() for c in row_as_string)

            try:
                if numbers / len(row_as_string) > self.pct_digit:
                    row_types.append(True)
                else:
                    row_types.append(False)
            except ZeroDivisionError:
                row_types.append(False)

        return row_types

    def ids_from_wide_similarities(
        self,
        similarities: np.ndarray,
        fuzzy_matches: np.ndarray,
    ) -> List[str]:
        """
        Get column ids from wide similarity and fuzzy
        match matrices.

        Args:
            similarities (np.ndarray): Similarity matrix of shape N x M.
            fuzzy_matches (np.ndarray): Match matrix of shape N x M.
        Returns:
            List[str]: Column ids.
        """
        column_match = self.stable_match_wide(similarities * fuzzy_matches)
        scores = np.take_along_axis(
            similarities * fuzzy_matches,
            np.expand_dims(column_match, 1),
            axis=1,
        ).squeeze()

        # Array of length M, indicates position of columns IF1
        # to IF11 (in that order) in the extracted array
        column_match = np.where(scores, column_match, -1)

        column_ids = [""] * similarities.shape[0]
        for i, j in enumerate(column_match):
            if j >= 0:
                index = self.colnames_out[j]
                column_ids[i] = index

        return column_ids

    def ids_from_long_similarities(
        self,
        similarities,
        fuzzy_matches,
    ):
        """
        Get column ids from wide similarity and fuzzy
        match matrices.

        Args:
            similarities (np.ndarray): Similarity matrix of shape N x M.
            fuzzy_matches (np.ndarray): Match matrix of shape N x M.
        Returns:
            List[str]: Column ids.
        """
        column_match = self.stable_match_long(similarities * fuzzy_matches)
        scores = np.take_along_axis(
            similarities * fuzzy_matches,
            np.expand_dims(column_match, 0),
            axis=0,
        )

        # Array of length M, indicates position of columns IF1
        # to IF11 (in that order) in the extracted array
        column_match = np.where(scores, column_match, -1).squeeze()

        column_ids = [""] * similarities.shape[0]
        for i, j in enumerate(column_match):
            if j >= 0:
                index = self.colnames_out[i]
                column_ids[j] = index

        return column_ids
    
    @staticmethod
    def stable_match_long(similarities: np.array) -> np.array:
        """
        From a 2D matrix of similarities between the column names
        desired in the output and the column names in an input matrix,
        returns a stable matching.

        Args:
            similarities (np.array): Similarity matrix.
        """
        order = similarities.argsort(0)
        ncolumns_to_match = order.shape[1]
        ncolumns = order.shape[0]

        FREE = -1
        match = FREE * np.ones(ncolumns_to_match, dtype=np.int_)
        jnext = ncolumns * np.ones(ncolumns_to_match, dtype=np.int_)
        rev_match = FREE * np.ones(ncolumns, dtype=np.int_)

        while np.any(match == FREE):
            i = np.where(match == FREE)[0][0]
            jnext[i] -= 1
            j = order[jnext[i], i]
            if rev_match[j] == FREE:
                rev_match[j], match[i] = i, j
            else:
                if similarities[j, rev_match[j]] < similarities[j, i]:
                    match[rev_match[j]] = FREE
                    rev_match[j], match[i] = i, j

        return match

    @staticmethod
    def stable_match_wide(similarities: np.array) -> np.array:
        """
        From a 2D matrix of similarities between the column names
        desired in the output and the column names in an input matrix,
        returns a stable matching.

        Args:
            similarities (np.array): Similarity matrix.
        """
        transposed_similarities = similarities.T

        order = transposed_similarities.argsort(0)
        ncolumns_to_match = order.shape[1]
        ncolumns = order.shape[0]

        FREE = -1
        match = FREE * np.ones(ncolumns_to_match, dtype=np.int_)
        jnext = ncolumns * np.ones(ncolumns_to_match, dtype=np.int_)
        rev_match = FREE * np.ones(ncolumns, dtype=np.int_)

        while np.any(match == FREE):
            i = np.where(match == FREE)[0][0]
            jnext[i] -= 1
            j = order[jnext[i], i]
            if rev_match[j] == FREE:
                rev_match[j], match[i] = i, j
            else:
                if transposed_similarities[j, rev_match[j]] < transposed_similarities[j, i]:
                    match[rev_match[j]] = FREE
                    rev_match[j], match[i] = i, j

        return match

    @staticmethod
    def similar(a: str, b: str) -> float:
        """
        Returns the Levenshtein similarity between two strings given as inputs.

        Args:
            a (str): First string.
            b (str): Second string.
        """
        return SequenceMatcher(None, a, b).ratio()


In [143]:
import re

def replace_comma(col):
    return col.str.replace(",", r"\.")


def clear_dates(s):
    if isinstance(s, str):
        pattern = r'\b\d{1,2}\-([A-Za-z_À-ÿ]{3}|\d{1,2})(?:\.)?\-\d{2,4}\b'
        return re.sub(pattern, '', s)
    else:
        return s


def clean_string(s):
    if isinstance(s, str):
        pattern = r'[^0-9\.\-]'
        return re.sub(pattern, '', s)
    else:
        return s


def cast_column(col):
    return col.astype(str).str.strip().astype(float)


def extract_first_number(s):
    if isinstance(s, str):
        pattern = r'-?\d+(\.\d+)?'
        match = re.search(pattern, s)

        if match:
            # Convert the matched string to a numeric value
            first_number = match.group()
            return first_number
        return s
    else:
        return s

In [144]:
cleaner = NewTableCleaner(fields, regexes)
clean_tables = {}

for siren, table_list in extracted_tables.items():
    selected_df = None
    num_cols = 0
    
    for df in table_list:
        # Clean
        clean_df = cleaner.clean_df(df)
        if clean_df is not None:
            first_column = clean_df.iloc[:, 0]
            df_to_filter = clean_df.iloc[:, 1:]

            filtered_df = df_to_filter.loc[:, ~df_to_filter.columns.isin([""])]
            if filtered_df.shape[1] > num_cols:
                num_cols = filtered_df.shape[1]
                # Clean up les colonnes numériques
                filtered_df = filtered_df.apply(replace_comma)
                filtered_df = filtered_df.applymap(clear_dates)
                filtered_df = filtered_df.applymap(clean_string)
                filtered_df = filtered_df.replace("", "0")
                filtered_df = filtered_df.replace(".", "0")
                filtered_df = filtered_df.replace('\.\.', '.', regex=True)
                filtered_df = filtered_df.applymap(extract_first_number)
                filtered_df = filtered_df.apply(cast_column)
                filtered_df = filtered_df.fillna(0.0)
                selected_df = pd.concat([first_column, filtered_df], axis=1)
        
    clean_tables[siren] = selected_df

---
306769688
0
5
---
307299248
0
0
hi
1
5
---
345039416
---
345039416
0
0
hi
1
0
hi
2
0
hi
3
0
hi
4
0
hi
5
0
hi
6
0
hi
7
0
hi
8
0
hi
---
379984891
---
379984891
---
379984891
0
0
hi
---
380129866
0
5
---
384518114
0
2
---
409759156
0
4
---
430107359
---
430107359
0
5
---
432668432
---
432668432
0
5
---
440419240
0
0
hi
1
3
---
500413505
0
0
hi
---
500413505
0
5
---
500440813
0
0
hi
---
500440813
0
5
---
501614572
---
501614572
0
4


In [148]:
idx = 5
siren = siren_list[idx]
siren

'384518114'

In [149]:
extracted_tables[siren][0]

,col,col,col,col,col,col,col,col,col,col,col
line_id,,,,,,,,,,,
1,H-déc-21,NaN,NaN,RENSEIGNEMENTS,RENSEIGNEMENTS CONCERNANT,LES FILLALES,ET LES,LES PARTICIPATIONS,on devises de la,la fillale,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{Code de commerce,commerce R 123-197-2*},NaN,NaN
3,Informetions financières,NaN,Copta,NaN,NaN,NaN,Re,NaN,NaN,NaN,NaN
4,NaN,NaN,SoclalRéserves etreport à nouveau avant {en af...,Quots-Part du capital détenue {en pourcentage),NaN,NaN,urnes remboursé,Coutions etes donnés par ta sochété,Chiffre d'affaires HT du damier exercice écoulé,Réauitat OMdend (bénéfice ou ports du dernier ...,Obser. OMdend vations Hate de encaissé clôture}
5,NaN,NaN,NaN,NaN,NaN,NaN,|,|,NaN,NaN,NaN
6,A. Renseignements détaillés concemant les flll...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,et participations cl-desous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1. Flales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,(+ 50% du capital détenu par la société),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
clean_tables[siren]

,,Résultats (bénéfices ou pertes à 100%) du dernier exercice clos (en K devises),Quote-part du capital détenue (en %)
line_id,,,
11,Siren: 790 212 351,1000.0,100.00
12,"BD LIFESCIENCES France (153rue de Courcelles, ...",0.0,0.00
13,[790 953 384,39100.0,100.00
14,"BOU (153 rue de Courcelles, Paris) Siren : 381...",1150.0,100.00
15,"BD France (153 rue de Courcelles, Paris) Siren...",-11300.0,100.00
16,"ÆOLAS (8 rue Voltaire, Grenoble) Siren : 382 1...",7482300.0,100.00
17,"SCI MANGIN (Rue du général Mangin, Grenoble) S...",0.0,0.00
18,980,327000.0,66.67
19,"METAPHORA (153 rue de Courcelles, Paris) Siren...",312758.0,86.44
